# Discover & Process Sentinel-1 InSAR pairs

- The provider used for this notebook is `cop_dataspace`
- A free account needs to be created at https://dataspace.copernicus.eu/
- The credentials need to be configured as explained here https://eodag.readthedocs.io/en/stable/getting_started_guide/configure.html

In [ ]:
%load_ext autoreload
%autoreload 2

from folium import Map, LayerControl

from eo_tools.util import explore_products
from eo_tools.util import show_insar_coh, show_insar_phi, show_sar_int, show_cog
from eo_tools.S1 import process_InSAR
import geopandas as gpd
import folium
import folium.plugins
import logging
import os

from eodag import EODataAccessGateway

# credentials need to be stored in the following file (see EODAG docs)
confpath = "/data/eodag_config.yml"
dag = EODataAccessGateway(user_conf_file_path=confpath)
# make sure cop_dataspace will be used 
dag.set_preferred_provider("cop_dataspace")
logging.basicConfig(level=logging.INFO)

## Search

In [ ]:
# load a geometry
shp = gpd.read_file("../data/Kilauea.geojson").iloc[0].geometry
search_criteria = {
    # "productType": "S1_SAR",
    "productType": "S1_SAR_SLC",
    "start": "2018-04-11",
    "end": "2018-05-30",
    "geom": shp,
}
results, _ = dag.search(**search_criteria)

## Visualize by groups

- For InSAR, products must be overlapping and from the same relative orbit.
- They also have to be of the same ascending / descending configuration.

In [ ]:
# TODO: add filter on ascending / descending
explore_products(results, shp)

## Download

In [ ]:
# fill indices of products to download
# indices_to_download = [12, 11]
indices_to_download = [4, 22]
to_download = [results[i] for i in indices_to_download]

# if the data has already been downloaded, just re-run the cell to get the product paths
dl = dag.download_all(to_download, outputs_prefix="/data/S1/", extract=False)

# Process

In [ ]:
# substr = ".SAFE/" 
# substr = ".SAFE/manifest.safe" 
# dl0 = dl[0].replace(".zip",substr)
# dl1 = dl[1].replace(".zip",substr) 
# print(dl1)

In [ ]:
# extract paths for processing
insar_dirs = process_InSAR(
    # dl1,
    dl[0],
    # dl0,
    dl[1],
    outputs_prefix="/data/res",
    tmp_dir="/data/tmp",
    aoi_name = "Kilauea-CD",
    shp=shp,
    pol="VH",
    coh_only=False,
    intensity=True,
    clear_tmp_files=True,
    resume=False
)

In [ ]:
insar_dir = "/data/res/S1_InSAR_VV_2018-04-11-161524__2018-05-29-161526_Kilauea-CD/"

In [ ]:
from eo_tools.dem import retrieve_dem
file_dem = "/data/res/dem_kilauea.tif"
# retrieve_dem(shp, file_dem)

In [ ]:
from eo_tools.S1 import change_int

file_mst = f"{insar_dir}/int_mst.tif"
file_slv = f"{insar_dir}/int_slv.tif"
file_change = f"{insar_dir}/change.tif"
# change_int(file_slv, file_mst, file_change)

In [ ]:
m = Map()
tile = folium.TileLayer(
        tiles = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
        attr = 'Esri',
        name = 'Esri Satellite',
        overlay = False,
        control = True
       ).add_to(m)
# show_cog("/data/kil_2019dem_hs.tif", m)
show_cog(file_dem, m, rescale="800,1800")#, algorithm="hillshade")
show_cog(f"{insar_dir}/coh.tif", m, rescale="0,1")
show_cog(f"{insar_dir}/change.tif", m, rescale="-1,1", colormap_name="blues_r")
show_cog(f"{insar_dir}/int_mst.tif", m, rescale="0,0.4")
show_cog(f"{insar_dir}/int_slv.tif", m, rescale="0,0.4")
LayerControl().add_to(m)
m